# StrOutputParser란 무엇인가?

---

## 1. StrOutputParser의 역할

- **LangChain LCEL(파이프라인) 체인에서 LLM의 응답(출력)을 일반 Python 문자열(`str`)로 변환하는 전용 파서(후처리 모듈)**입니다.
- LLM API의 표준 응답 객체는 `AIMessage`와 같이 구조화된 객체(`content` 속성 등)를 포함할 수 있는데,
- StrOutputParser를 파이프라인 마지막에 연결하면 **자동으로 최종 결과를 `str`로 변환**해줍니다.
- 이로 인해 파이썬 코드, 함수, 사용자 응답 등에서 **직접 문자열처럼 활용**할 수 있습니다.

---

## 2. 동작 구조 및 원리

- **기본 체인 출력 흐름:**  
  - `prompt | llm` → `AIMessage` (혹은 `ChatMessage` 등 객체 반환)
- **StrOutputParser 추가:**  
  - `prompt | llm | StrOutputParser()` → `str`(문자열) 반환
- **내부 동작:**  
  - LLM의 출력(`AIMessage`)에서 `.content` 또는 `.text` 등 **핵심 텍스트 부분만 추출하여 반환**합니다.

---

## 3. 실무적 효용 및 장점

- **코드 간결화:**  
  - 체인 결과를 별도의 객체 속성(`.content`)으로 일일이 꺼내지 않고, **바로 문자열처럼 활용 가능**
- **타 시스템/함수 연동 편의성:**  
  - 단순 문자열을 반환하므로, 파일 저장/로그/데이터베이스 입력/웹 응답 등에 즉시 사용 가능
- **복잡한 출력 타입 처리 자동화:**  
  - OpenAI 등 다양한 LLM 라이브러리와 연동 시, 결과 타입이 다를 때도 **일관된 문자열 반환 보장**

In [6]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
import os

# 환경변수(.env) 로드 및 API키 확인
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError(".env 파일에 GOOGLE_API_KEY 를 반드시 입력하세요.")

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(
    "'{product}'에 대한 창의적인 마케팅 슬로건을 3개 만들어주세요."
)

# Gemini LLM 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",      # gemini-1.5-flash, gemini-pro, gemini-1.5-pro 등
    google_api_key=api_key,
    temperature=0.8
)

# LCEL 파이프라인 구성
chain = prompt | llm | StrOutputParser()

# 체인 실행 및 결과 출력
result = chain.invoke({"product": "친환경 텀블러"})
print(result)

# 또는 직접 문자열로도 가능
simple_chain = prompt | llm
result2 = simple_chain.invoke({"product": "스마트 워치"})
print(result2)

1. **지구를 위한 작은 실천, 당신의 스타일을 위한 완벽한 선택: 친환경 텀블러.** (실용성과 개성을 동시에 강조)

2. **플라스틱 걱정은 이제 그만, 스타일과 지구를 동시에 담다: 친환경 텀블러.** (문제 해결과 긍정적 결과를 제시)

3. **오늘부터, 당신의 손안에 담긴 지속가능한 미래: 친환경 텀블러.** (감성적 어필과 미래지향적인 메시지 전달)
content='1. **당신의 손목 위의 미래, 지금 만나보세요. (Your future, on your wrist. Experience it now.)** -  미래지향적이고, 제품의 첨단 기술을 강조합니다.\n\n2. **시간을 넘어, 당신의 삶을 연결하세요. (Beyond time, connect your life.)** -  스마트워치가 단순한 시계를 넘어 라이프스타일과 연결되는 도구임을 표현합니다.\n\n3. **손목 위의 작은 변화, 삶의 큰 가능성. (A small change on your wrist, a big possibility in your life.)** -  스마트워치의 편리함과 그로 인한 삶의 질 향상을 간결하게 보여줍니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-4ac6adbf-b82c-41a7-8a55-90138cbfe5bd-0' usage_metadata={'input_tokens': 28, 'output_tokens': 181, 'total_tokens': 209, 'input_token_details': {'cache_read': 0}}
